In [ ]:
import arcpy
import sys
sys.path.append("../src")
from utils import Utils
from multiscaleAlignMasking import MultiscaleAlignMasking
from processDem import ProcessDem
import os

# 1. Align all rasters, including backscatter and higher resolution files to the 1-m DEM

In [ ]:
# ================= CONFIGURATION =================
REEF_NAME = "ElkRapids"  # Name of the reef directory
ABRV = "ER"
BASE_DIR = fr"C:\Users\ageglio\Documents\NLM_DataRelease\Updated_DataRelease_Surfaces_NewProducts"

# List resolutions in priority order (first valid one found becomes the reference/snap raster) 
### !!!TARGET RESOLUTIONS MUST MATCH THE SUBFOLDER AND FILE NAME!!!!
TARGET_RESOLUTIONS = ["1m", "0.5m", "0.25m"]

# Define markers used in filenames
DEM_MARKER = "BY"  # Bathymetry
BS_MARKER = "BS"   # Backscatter

PRODUCTS = ["slope", "aspect", "roughness", "tpi", "tri", "hillshade", "shannon", "bathymorphons", "idw"]

DIVISIONS = None # Divide the height by this to run tile processing of DEM file, automatic overalap computed.

# ================ INITIALIZE INPUTS ================
# --- 1. Identify Inputs ---
REEF_DIR = os.path.join(BASE_DIR, REEF_NAME)
INPUT_DEMS = Utils.find_rasters(REEF_DIR, ABRV, TARGET_RESOLUTIONS, DEM_MARKER)
INPUT_BSS = Utils.find_rasters(REEF_DIR, ABRV, TARGET_RESOLUTIONS, BS_MARKER)

# --- 2. Basic validation ---
if not INPUT_DEMS:
    raise FileNotFoundError(f"No DEMs found in {REEF_DIR} for resolutions {TARGET_RESOLUTIONS}")
print(f"PRODUCTS: {PRODUCTS}")
# =================================================

In [ ]:
# ================= ALIGNMENT =================
# --- Align all rasters and return mask intersection ---
aligned_dems, aligned_bss, intersection_mask = MultiscaleAlignMasking.align_rasters_return_mask(INPUT_DEMS, INPUT_BSS)
# =================================================

# 2. Generate derived DEM products

In [ ]:
# Generate terrain products of all of the dems and align the backscatter files
for input_dem, input_bs in zip(INPUT_DEMS, INPUT_BSS):
    # Create an instance of the ProcessDEM class with the specified parameters
    generateHabitatDerivates = ProcessDem(
                                    input_dem=input_dem,
                                    input_bs=input_bs,
                                    binary_mask=intersection_mask,
                                    divisions=DIVISIONS,  # Divide the height by this to run tile processing
                                    shannon_window=[3, 9, 21],
                                    products=PRODUCTS,
                                    fill_method="IDW",
                                    fill_iterations=1,
                                    water_elevation=183.6,
                                    keep_chunks=False,
                                    bypass_mosaic=False,
                                    )
    # Process the DEM and generate the habitat derivatives
    generateHabitatDerivates.process_dem()

In [ ]:
# allowing the user to run all without executing the trackline portion
assert 1==2

# 3. Tracklines to shapefile

In [ ]:
import sys
import os
sys.path.append("../src")
from extents import GetExtents

# Define the output folder path for shapefiles
out_folder = "..\\shapefiles"
os.makedirs(out_folder, exist_ok=True)

trackline_folder = r"C:\Users\ageglio\Documents\NLM_DataRelease\FINAL_DataRelease_Files_21-23\BayHarborReef_2021 (1)\BayHarborReef_2021\Bay Harbor Reef_Tracklines"
print("folder path chosen: ", trackline_folder)

# combine and convert tracklines to a shapefile
GetExtents.create_tracklines(trackline_folder, out_folder, utm_zone=16)